## Search Engine With Tools and Agent

In [1]:
# arxiv-- reasearch
#wikipidia -- feaching data

from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [ ]:
## now we use Wekipedia and arxiv wrapper to featch data from wekipedia and arxiv
# Wrapper around WikipediaAPI.

# To use, you should have the wikipedia python package installed. This wrapper will use the 
# Wikipedia API to conduct searches and fetch page summaries. By default, 
# it will return the page summaries of the top-k results.
# It limits the Document content by doc_content_chars_max
api_wikipedia_wraper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wikipedia_wraper) # we use this to see the content form wikipedia now similarly for arxi below
wiki.name

#so this is the step first we create wrapper -- query --- wrap

'wikipedia'

In [4]:
#similarly for arxiv
api_arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
axiv=ArxivQueryRun(api_wrapper=api_arxiv_wrapper)
axiv.name

'arxiv'

In [5]:
tools=[wiki,axiv]


In [6]:
# costome tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
import os
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\Komal Patil\AppData\Local\Temp\ipykernel_15800\2748617229.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
d:\sudesh\Gerative AI\End to end Q and A app with GenAI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
document=RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50).split_documents(docs)
vectordb=FAISS.from_documents(documents=document, embedding=embedding)
retrive=vectordb.as_retriever()
retrive

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AD31E1FEE0>, search_kwargs={})

In [13]:
from langchain.tools.retriever import create_retriever_tool
retriever_tools=create_retriever_tool(retriever=retrive, name="langsmith-search", description="Search any infomation about langsmith ")
retriever_tools.name

'langsmith-search'

In [14]:
tools=[axiv, wiki, retriever_tools]

In [15]:
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\sudesh\\Gerative AI\\End to end Q and A app with GenAI\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any infomation about langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001AD320EA200>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS

In [16]:
#Run all the tools with Agents and LLMs
from dotenv import load_dotenv
load_dotenv()
import google.generativeai as genai
from langchain_groq import ChatGroq
#for loading the the file content or api we use
import os
os.environ['GROQ_TESTING_API']=os.getenv('GROQ_TESTING_API')
groq_api_key=os.getenv('GROQ_TESTING_API')
genai.api_key=os.getenv('GOOGLE_API_KEY')

llm=ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")


In [18]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [19]:
from langchain.agents import create_tool_calling_agent
agent=create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [20]:
from langchain.agents import AgentExecutor
agen_executor=AgentExecutor(agent=agent, tools=tools, verbose=True)
agen_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [21]:
#now see the magic below
agen_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with

LangSmith is a platform for building production-grade LLM applications.

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or withoutLangSmith is a platform for building production-grade LLM applications. It's framework-agnostic, meaning it can be used with or without LangChain.  You can find more information about setting up LangSmith in the provided how-to guide. 


> Finished chain.


{'input': 'Tell me about Langsmith',
 'output': "LangSmith is a platform for building production-grade LLM applications. It's framework-agnostic, meaning it can be used with or without LangChain.  You can find more information about setting up LangSmith in the provided how-to guide. \n"}

As you can see it will not only search in docs as well as if you ask question it will search in wekipedia too that's amazing is it! and not only that by using hub you can able to take past history too.

In [ ]:
## now let us go to end to end project

